In [326]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import random
from sklearn.preprocessing import StandardScaler
import plotly.express as px


In [327]:
final_df = pd.read_csv('../data/final_V2.csv')
final_df 

,Name,Countries,Genres,Budget(USD)_Inflated,Domestic(USD)_Inflated,Domestic_Percentage,Foreign(USD)_Inflated,Foreign_Percentage,Worldwide(USD)_Inflated,Runtime(mins),...,animation,teen,film adaptation,musical,history,coming of age,sports,black comedy,war film,Worldwide_profit
0,10 cloverfield lane,united states of america,"drama, horror, sci-fi, thriller",6.076746e+06,8.760602e+07,0.654010,4.634613e+07,0.345990,1.339521e+08,103,...,0,0,0,0,0,0,0,0,0,22.043400
1,"10,000 bc",united states of america,"thriller, adventure, costume drama, action, ad...",1.441563e+08,1.301309e+08,0.351333,2.402606e+08,0.648667,3.703914e+08,109,...,0,0,0,0,0,0,0,0,0,2.569373
2,12 rounds,united states of america,"action thrillers, thriller, action, crime fiction",2.726275e+07,1.667757e+07,0.708013,6.877890e+06,0.291987,2.355546e+07,108,...,0,0,0,0,0,0,0,0,0,0.864016
3,12 strong,united states of america,"action, drama, history, war",3.979050e+07,5.209113e+07,0.644274,2.876133e+07,0.355726,8.085246e+07,130,...,0,0,0,0,1,0,0,0,0,2.031954
4,12 years a slave,united states of america,"biography, drama, history",2.539260e+07,7.195247e+07,0.301875,1.663993e+08,0.698125,2.383517e+08,134,...,0,0,0,0,1,0,0,0,0,9.386660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,zoolander,united states of america,"parody, comedy of errors, comedy",4.851315e+07,7.826600e+07,0.743197,2.704388e+07,0.256803,1.053099e+08,90,...,0,0,0,0,0,0,0,0,0,2.170749
2685,zoolander 2,united states of america,"action, adventure, comedy, mystery, romance",6.076746e+07,3.506124e+07,0.508592,3.387664e+07,0.491408,6.893788e+07,101,...,0,0,0,0,0,0,0,0,0,1.134454
2686,zoom,united states of america,"science fiction, action, fantasy, adventure",5.014200e+07,1.717625e+07,0.958658,7.407176e+05,0.041342,1.791697e+07,93,...,0,0,0,0,0,0,0,0,0,0.357325
2687,zootopia,united states of america,"action, adventure, animation, comedy, crime, f...",1.823024e+08,4.147601e+08,0.333340,8.294952e+08,0.666660,1.244255e+09,108,...,1,0,0,0,0,0,0,0,0,6.825228


In [328]:
final_df.columns

Index(['Name', 'Countries', 'Genres', 'Budget(USD)_Inflated',
       'Domestic(USD)_Inflated', 'Domestic_Percentage',
       'Foreign(USD)_Inflated', 'Foreign_Percentage',
       'Worldwide(USD)_Inflated', 'Runtime(mins)', 'Rating', 'Audience_Score',
       'Critics_Score', 'plot_summary', 'Month', 'Day', 'Year',
       'Foreign_higher', 'emotion', 'emotion_score', 'drama', 'comedy',
       'thriller', 'action', 'adventure', 'romance film', 'horror', 'fantasy',
       'mystery', 'crime fiction', 'indie', 'biography', 'family film',
       'sci-fi', 'period piece', 'animation', 'teen', 'film adaptation',
       'musical', 'history', 'coming of age', 'sports', 'black comedy',
       'war film', 'Worldwide_profit'],
      dtype='object')

## preparing the data

In [329]:
# Dropping columns deemed irrelevant for the model
df_ols = final_df.drop(columns=["Genres", "Day","Name", "plot_summary", "Countries", "Worldwide(USD)_Inflated", "Domestic(USD)_Inflated", "Foreign(USD)_Inflated", "Worldwide(USD)_Inflated", "Foreign_higher"])

In [330]:
df_ols

,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Rating,Audience_Score,Critics_Score,Month,Year,emotion,...,animation,teen,film adaptation,musical,history,coming of age,sports,black comedy,war film,Worldwide_profit
0,6.076746e+06,0.654010,0.345990,103,pg-13,0.79,0.91,Mar,2016,fear,...,0,0,0,0,0,0,0,0,0,22.043400
1,1.441563e+08,0.351333,0.648667,109,pg-13,0.37,0.10,Mar,2008,fear,...,0,0,0,0,0,0,0,0,0,2.569373
2,2.726275e+07,0.708013,0.291987,108,pg-13,0.45,0.31,Mar,2009,anger,...,0,0,0,0,0,0,0,0,0,0.864016
3,3.979050e+07,0.644274,0.355726,130,r,0.62,0.50,Jan,2018,fear,...,0,0,0,0,1,0,0,0,0,2.031954
4,2.539260e+07,0.301875,0.698125,134,r,0.90,0.95,Oct,2013,sadness,...,0,0,0,0,1,0,0,0,0,9.386660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,4.851315e+07,0.743197,0.256803,90,pg-13,0.80,0.64,Sep,2001,anger,...,0,0,0,0,0,0,0,0,0,2.170749
2685,6.076746e+07,0.508592,0.491408,101,pg-13,0.20,0.22,Feb,2016,surprise,...,0,0,0,0,0,0,0,0,0,1.134454
2686,5.014200e+07,0.958658,0.041342,93,pg,0.34,0.05,Aug,2006,neutral,...,0,0,0,0,0,0,0,0,0,0.357325
2687,1.823024e+08,0.333340,0.666660,108,pg,0.92,0.98,Mar,2016,neutral,...,1,0,0,0,0,0,0,0,0,6.825228


In [331]:
# Standardizing 

columns_to_scale = ['Budget(USD)_Inflated', 'Domestic_Percentage', 'Foreign_Percentage', 'Runtime(mins)', 'Audience_Score', 'Critics_Score', 'emotion_score']
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_ols[columns_to_scale])
df_ols[columns_to_scale] = scaled_data

df_ols

,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Rating,Audience_Score,Critics_Score,Month,Year,emotion,...,animation,teen,film adaptation,musical,history,coming of age,sports,black comedy,war film,Worldwide_profit
0,-0.820351,0.391747,-0.391747,-0.291947,pg-13,1.012762,1.412014,Mar,2016,fear,...,0,0,0,0,0,0,0,0,0,22.043400
1,1.305928,-0.877514,0.877514,0.052120,pg-13,-1.230673,-1.539543,Mar,2008,fear,...,0,0,0,0,0,0,0,0,0,2.569373
2,-0.494109,0.618207,-0.618207,-0.005225,pg-13,-0.803352,-0.774325,Mar,2009,anger,...,0,0,0,0,0,0,0,0,0,0.864016
3,-0.301195,0.350920,-0.350920,1.256351,r,0.104705,-0.081984,Jan,2018,fear,...,0,0,0,0,1,0,0,0,0,2.031954
4,-0.522908,-1.084916,1.084916,1.485729,r,1.600328,1.557770,Oct,2013,sadness,...,0,0,0,0,1,0,0,0,0,9.386660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,-0.166876,0.765751,-0.765751,-1.037423,pg-13,1.066177,0.428161,Sep,2001,anger,...,0,0,0,0,0,0,0,0,0,2.170749
2685,0.021828,-0.218058,0.218058,-0.406635,pg-13,-2.138730,-1.102276,Feb,2016,surprise,...,0,0,0,0,0,0,0,0,0,1.134454
2686,-0.141793,1.669279,-1.669279,-0.865390,pg,-1.390919,-1.721738,Aug,2006,neutral,...,0,0,0,0,0,0,0,0,0,0.357325
2687,1.893336,-0.952969,0.952969,-0.005225,pg,1.707158,1.667086,Mar,2016,neutral,...,1,0,0,0,0,0,0,0,0,6.825228


In [332]:
# one-hot encoding rating column, dropping the first column in order to avoid introducing multicollinearity
df_ols = pd.get_dummies(df_ols, columns=['Rating'], drop_first=True)
df_ols = df_ols.drop(columns="Rating_nc-17") # Only 5 nc-17 movies, not considered for analysis
# one-hot encoding month column, dropping first column for same reason
df_ols = pd.get_dummies(df_ols, columns=['Month'], drop_first=True)
# same thing for the emotion column
df_ols = pd.get_dummies(df_ols, columns=['emotion'], drop_first=True)


In [333]:
df_ols

,Budget(USD)_Inflated,Domestic_Percentage,Foreign_Percentage,Runtime(mins),Audience_Score,Critics_Score,Year,emotion_score,drama,comedy,...,Month_May,Month_Nov,Month_Oct,Month_Sep,emotion_disgust,emotion_fear,emotion_joy,emotion_neutral,emotion_sadness,emotion_surprise
0,-0.820351,0.391747,-0.391747,-0.291947,1.012762,1.412014,2016,0.050657,1,0,...,False,False,False,False,False,True,False,False,False,False
1,1.305928,-0.877514,0.877514,0.052120,-1.230673,-1.539543,2008,0.504088,1,0,...,False,False,False,False,False,True,False,False,False,False
2,-0.494109,0.618207,-0.618207,-0.005225,-0.803352,-0.774325,2009,0.056938,0,0,...,False,False,False,False,False,False,False,False,False,False
3,-0.301195,0.350920,-0.350920,1.256351,0.104705,-0.081984,2018,1.226752,1,0,...,False,False,False,False,False,True,False,False,False,False
4,-0.522908,-1.084916,1.084916,1.485729,1.600328,1.557770,2013,0.964755,1,0,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,-0.166876,0.765751,-0.765751,-1.037423,1.066177,0.428161,2001,-0.726523,0,1,...,False,False,False,True,False,False,False,False,False,False
2685,0.021828,-0.218058,0.218058,-0.406635,-2.138730,-1.102276,2016,1.495326,0,1,...,False,False,False,False,False,False,False,False,False,True
2686,-0.141793,1.669279,-1.669279,-0.865390,-1.390919,-1.721738,2006,0.055972,0,0,...,False,False,False,False,False,False,False,True,False,False
2687,1.893336,-0.952969,0.952969,-0.005225,1.707158,1.667086,2016,0.256964,0,1,...,False,False,False,False,False,False,False,True,False,False


## Defining features and labels for the 4 different models:
Labels: Foreign/domestic percentage for prediction
Predictors: before and after 2010

In [334]:
# Movies before 2010
df_before_2010 = df_ols[df_ols['Year'] < 2010]

# Movies including and after 2010
df_2010_and_after = df_ols[df_ols['Year'] >= 2010]

Function to train OLS model

In [335]:
def train_ols_model(dataframe, target_column, drop_columns):
    """
    Trains an OLS regression model.

    Parameters:
        dataframe (pd.DataFrame): The input DataFrame containing features and the target variable.
        target_column (str): The name of the column to be used as the target variable.
        drop_columns (list): List of columns to drop from the DataFrame.

    Returns:
        ols_model (sm.OLS): The trained OLS model.
    """
    # Defining features and labels
    y = dataframe[target_column]
    X = dataframe.drop(columns=drop_columns)

    # Adding a constant column for the OLS model
    X = sm.add_constant(X)

    # Training the OLS model
    feature_names = list(X.columns)
    label_name = target_column

    # Ensure predictors and target are numeric
    X = pd.DataFrame(X, columns=feature_names, dtype=float)
    y = pd.Series(y, name=label_name, dtype=float) # Including the label name

    # Fit the OLS model
    ols_model = sm.OLS(y, X).fit()
    return ols_model

Function to save the feature statistics for later use

In [336]:
import pandas as pd

def extract_feature_statistics(ols_model, output_csv_path=None):
    """
    Extracts feature statistics from an OLS regression model.

    Parameters:
        ols_model (sm.OLS): The trained OLS regression model.
        output_csv_path (str, optional): Path to save the feature statistics as a CSV file. Defaults to None.

    Returns:
        feature_stats (pd.DataFrame): A DataFrame containing the feature statistics.
    """
    # Extracting feature statistics
    coefficients = ols_model.params  # Coefficients (coef)
    standard_errors = ols_model.bse  # Standard errors (std err)
    t_values = ols_model.tvalues     # t-values (t)
    p_values = ols_model.pvalues     # p-values (P>|t|)

    # Creating a DataFrame
    feature_stats = pd.DataFrame({
        "Feature": coefficients.index,
        "Coefficient": coefficients.values,
        "Std_Error": standard_errors.values,
        "t_value": t_values.values,
        "p_value": p_values.values
    })

    # Saving the DataFrame to a CSV file if the path is provided
    if output_csv_path:
        feature_stats.to_csv(output_csv_path, index=False)

    return feature_stats

### 1st model: Foreign Percentage prediction 

In [337]:
# Defining the target column as well as which columns to drop
target_column = "Foreign_Percentage"
drop_columns = ["Foreign_Percentage", "Domestic_Percentage"]
# Path to save CSV file containing the feature statistics
output_csv_path = "../src/data/Machine_learning_data/feature_stats_Foreign.csv"

# Train the model
ols_model = train_ols_model(df_ols, target_column, drop_columns)

# Extract feature statistics and save to CSV
feature_stats_Foreign = extract_feature_statistics(ols_model, output_csv_path)

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Foreign_Percentage   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.256
Method:                 Least Squares   F-statistic:                     19.15
Date:                Fri, 20 Dec 2024   Prob (F-statistic):          5.21e-143
Time:                        14:19:01   Log-Likelihood:                -3391.9
No. Observations:                2689   AIC:                             6888.
Df Residuals:                    2637   BIC:                             7194.
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  -61.6763 

### "2nd" model: Domestic Percentage prediction 

In [338]:
# Defining the target column as well as which columns to drop
target_column = "Domestic_Percentage"
drop_columns = ["Foreign_Percentage", "Domestic_Percentage"]
# Path to save CSV file containing the feature statistics
output_csv_path = "../src/data/Machine_learning_data/feature_stats_Domestic.csv"

# Train the model
ols_model = train_ols_model(df_ols, target_column, drop_columns)

# Extract feature statistics and save to CSV
feature_stats_domestic = extract_feature_statistics(ols_model, output_csv_path)

# Displaying the model summary
print(ols_model.summary())

                             OLS Regression Results                            
Dep. Variable:     Domestic_Percentage   R-squared:                       0.270
Model:                             OLS   Adj. R-squared:                  0.256
Method:                  Least Squares   F-statistic:                     19.15
Date:                 Fri, 20 Dec 2024   Prob (F-statistic):          5.21e-143
Time:                         14:19:01   Log-Likelihood:                -3391.9
No. Observations:                 2689   AIC:                             6888.
Df Residuals:                     2637   BIC:                             7194.
Df Model:                           51                                         
Covariance Type:             nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 

Only difference between the two models is the sign of the coeffecients. Subsquent Analysis will have only the foreign_percentage as a target column

### Before 2010, foreign prediction

In [339]:
# Defining the target column as well as which columns to drop
target_column = "Foreign_Percentage"
drop_columns = ["Foreign_Percentage", "Domestic_Percentage"]
# Path to save CSV file containing the feature statistics
output_csv_path = "../src/data/Machine_learning_data/feature_stats_pre_2010.csv"

# Train the model
ols_model = train_ols_model(df_before_2010, target_column, drop_columns)

# Extract feature statistics and save to CSV
feature_stats_pre_2010 = extract_feature_statistics(ols_model, output_csv_path)

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Foreign_Percentage   R-squared:                       0.287
Model:                            OLS   Adj. R-squared:                  0.256
Method:                 Least Squares   F-statistic:                     9.310
Date:                Fri, 20 Dec 2024   Prob (F-statistic):           5.43e-57
Time:                        14:19:01   Log-Likelihood:                -1459.9
No. Observations:                1232   AIC:                             3024.
Df Residuals:                    1180   BIC:                             3290.
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 -106.8019 

### Including and after 2010, foreign prediction

In [340]:
# Defining the target column as well as which columns to drop
target_column = "Foreign_Percentage"
drop_columns = ["Foreign_Percentage", "Domestic_Percentage"]
# Path to save CSV file containing the feature statistics
output_csv_path = "../src/data/Machine_learning_data/feature_stats_post_2010.csv"

# Train the model
ols_model = train_ols_model(df_2010_and_after, target_column, drop_columns)

# Extract feature statistics and save to CSV
feature_stats_post_2010 = extract_feature_statistics(ols_model, output_csv_path)

# Displaying the model summary
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:     Foreign_Percentage   R-squared:                       0.272
Model:                            OLS   Adj. R-squared:                  0.246
Method:                 Least Squares   F-statistic:                     10.30
Date:                Fri, 20 Dec 2024   Prob (F-statistic):           5.26e-66
Time:                        14:19:01   Log-Likelihood:                -1866.8
No. Observations:                1457   AIC:                             3838.
Df Residuals:                    1405   BIC:                             4112.
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    7.6724 

### Comparing the statistically significant coefficients

In [341]:
feature_stats_pre_2010.head(5)

,Feature,Coefficient,Std_Error,t_value,p_value
0,const,-106.801877,17.099916,-6.245754,5.875668e-10
1,Budget(USD)_Inflated,0.232545,0.037820,6.148716,1.067251e-09
2,Runtime(mins),0.001423,0.032650,0.043586,9.652418e-01
3,Audience_Score,-0.074239,0.034194,-2.171104,3.012207e-02
4,Critics_Score,0.132448,0.034046,3.890313,1.057072e-04


In [342]:
feature_stats_post_2010.head(5)

,Feature,Coefficient,Std_Error,t_value,p_value
0,const,7.672370,18.652919,0.411323,6.808986e-01
1,Budget(USD)_Inflated,0.268493,0.036059,7.445842,1.673180e-13
2,Runtime(mins),0.087977,0.034072,2.582098,9.920614e-03
3,Audience_Score,-0.219732,0.036764,-5.976786,2.880698e-09
4,Critics_Score,0.068687,0.035100,1.956906,5.055637e-02


Defining funciton that removes all the features with a P-value > 0.05

In [343]:
def filter_significant_features(feature_stats, p_value_threshold=0.05):

    # Filter the DataFrame based on the p-value threshold
    significant_features = feature_stats[feature_stats["p_value"] <= p_value_threshold]
    
    return significant_features

In [344]:
feature_stats_pre_2010 = filter_significant_features(feature_stats_pre_2010)
feature_stats_post_2010 = filter_significant_features(feature_stats_post_2010)

In [348]:
feature_stats_pre_2010

,Feature,Coefficient,Std_Error,t_value,p_value
0,const,-106.801877,17.099916,-6.245754,5.875668e-10
1,Budget(USD)_Inflated,0.232545,0.037820,6.148716,1.067251e-09
3,Audience_Score,-0.074239,0.034194,-2.171104,3.012207e-02
4,Critics_Score,0.132448,0.034046,3.890313,1.057072e-04
5,Year,0.053173,0.008533,6.231243,6.427620e-10
8,comedy,-0.237642,0.062929,-3.776356,1.670675e-04
12,romance film,0.187719,0.062190,3.018484,2.594950e-03
14,fantasy,0.169102,0.084518,2.000793,4.564348e-02
15,mystery,0.206684,0.091475,2.259465,2.403630e-02
22,animation,0.396553,0.143052,2.772099,5.657204e-03


In [349]:
feature_stats_post_2010

,Feature,Coefficient,Std_Error,t_value,p_value
1,Budget(USD)_Inflated,0.268493,0.036059,7.445842,1.673180e-13
2,Runtime(mins),0.087977,0.034072,2.582098,9.920614e-03
3,Audience_Score,-0.219732,0.036764,-5.976786,2.880698e-09
6,emotion_score,0.068308,0.024961,2.736614,6.285724e-03
8,comedy,-0.244581,0.062232,-3.930182,8.901036e-05
9,thriller,0.198654,0.067359,2.949164,3.239043e-03
10,action,0.166320,0.070550,2.357472,1.853635e-02
11,adventure,0.303609,0.075784,4.006251,6.492127e-05
22,animation,0.551085,0.148554,3.709662,2.156265e-04
28,sports,-0.475866,0.157239,-3.026393,2.519905e-03


Plotting the coeffecients for the statistically significant features

In [351]:
import plotly.express as px

df = feature_stats_post_2010
fig = px.bar(
    df, 
    x='Feature', 
    y='Coefficient',
    hover_data=['Std_Error', 't_value', 'p_value'], 
    color='Coefficient',
    labels={'Feature': 'Feature Names', 'Coefficient': 'Coefficient Value'},  # Correctly map labels
    height=400
)
fig.show()

In [350]:
import plotly.express as px

# Your dataset
df = feature_stats_pre_2010

# Sorting features by absolute coefficient value for better visualization (optional)
df_sorted = df.sort_values(by="Coefficient", key=abs, ascending=False)

# Creating the bar plot
fig = px.bar(
    df_sorted, 
    x='Feature', 
    y='Coefficient', 
    hover_data=['Std_Error', 't_value', 'p_value'], 
    color='Coefficient',
    labels={'Feature': 'Feature Names', 'Coefficient': 'Coefficient Value'},  # Map column names for display
    height=400
)

# Display the plot
fig.show()